In [26]:
import numpy as np
import pandas as pd

import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score       
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [27]:
#导入数据
data1 = pd.read_csv("voice_train.csv")
data2 = pd.read_csv("voice_test.csv")
train = data1
test = data2

In [28]:
#观察数据
train

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.132786,0.079557,0.119090,0.067958,0.209592,0.141634,1.932562,8.308895,0.963181,0.738307,...,0.132786,0.110132,0.017112,0.253968,0.298222,0.007812,2.726562,2.718750,0.125160,male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2530,0.146023,0.092525,0.183434,0.041747,0.224337,0.182590,1.384981,5.118927,0.948999,0.659825,...,0.146023,0.195640,0.039506,0.275862,0.533854,0.007812,2.992188,2.984375,0.258924,female
2531,0.131884,0.084734,0.153707,0.049285,0.201144,0.151859,1.762129,6.630383,0.962934,0.763182,...,0.131884,0.182790,0.083770,0.262295,0.832899,0.007812,4.210938,4.203125,0.161929,female
2532,0.142056,0.095798,0.183731,0.033424,0.224360,0.190936,1.876502,6.604509,0.946854,0.654196,...,0.142056,0.209918,0.039506,0.275862,0.494271,0.007812,2.937500,2.929688,0.194759,female
2533,0.143659,0.090628,0.184976,0.043508,0.219943,0.176435,1.591065,5.388298,0.950436,0.675470,...,0.143659,0.172375,0.034483,0.250000,0.791360,0.007812,3.593750,3.585938,0.311002,female


In [29]:
#预测性别是分类问题，将性别数据化
train['label'] = train['label'].map( {'male': 1, 'female': 0} ).astype(int)
train

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,1
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,1
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,1
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,1
4,0.132786,0.079557,0.119090,0.067958,0.209592,0.141634,1.932562,8.308895,0.963181,0.738307,...,0.132786,0.110132,0.017112,0.253968,0.298222,0.007812,2.726562,2.718750,0.125160,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2530,0.146023,0.092525,0.183434,0.041747,0.224337,0.182590,1.384981,5.118927,0.948999,0.659825,...,0.146023,0.195640,0.039506,0.275862,0.533854,0.007812,2.992188,2.984375,0.258924,0
2531,0.131884,0.084734,0.153707,0.049285,0.201144,0.151859,1.762129,6.630383,0.962934,0.763182,...,0.131884,0.182790,0.083770,0.262295,0.832899,0.007812,4.210938,4.203125,0.161929,0
2532,0.142056,0.095798,0.183731,0.033424,0.224360,0.190936,1.876502,6.604509,0.946854,0.654196,...,0.142056,0.209918,0.039506,0.275862,0.494271,0.007812,2.937500,2.929688,0.194759,0
2533,0.143659,0.090628,0.184976,0.043508,0.219943,0.176435,1.591065,5.388298,0.950436,0.675470,...,0.143659,0.172375,0.034483,0.250000,0.791360,0.007812,3.593750,3.585938,0.311002,0


In [30]:
#分出特征值和标签
X = train.drop("label", axis=1)
y = train['label']

In [31]:
#划分训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=1, train_size=0.8)

In [32]:
#特征值归一化处理
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)

In [33]:
scaler.fit(x_test)
x_test = scaler.transform(x_test)

In [34]:
scaler.fit(test)
test = scaler.transform(test)

In [35]:
results = pd.DataFrame()
columns = ["SVM", "DecisionTree", "LogisticRegression"]

In [36]:
svc = SVC(kernel='linear', C = 1)
svc.fit(x_train, y_train)
svc_pre = svc.predict(x_test)

In [37]:
decision_tree_classifier = DecisionTreeClassifier()
param = {'criterion':['gini', 'entropy'], 'max_depth':[2, 4, 6, 8, 10], 'min_samples_leaf':[2, 3, 5, 10], 'min_impurity_decrease':[0.1, 0.2, 0.5]}
grid = GridSearchCV(DecisionTreeClassifier(), param_grid=param, cv=6)
grid.fit(x_train, y_train)
print('最优分类器:',grid.best_params_,'最优分数:', grid.best_score_)  # 得到最优的参数和分值

最优分类器: {'criterion': 'gini', 'max_depth': 2, 'min_impurity_decrease': 0.1, 'min_samples_leaf': 2} 最优分数: 0.9511834319526628


In [38]:
dt = DecisionTreeClassifier(criterion = 'gini', max_depth = 2, min_impurity_decrease = 0.1, min_samples_leaf = 2)
dt.fit(x_train, y_train)
dt_pre = dt.predict(x_test)

In [39]:
lg = LogisticRegression(penalty = 'l1',solver='liblinear') 
lg.fit(x_train, y_train)
lg_pre = lg.predict(x_test)

In [40]:
results_append = pd.DataFrame(data=[[svc.score(x_train, y_train), dt.score(x_train, y_train), lg.score(x_train, y_train)]], columns=columns)
results = results.append(results_append, ignore_index = True)

In [41]:
results_append = pd.DataFrame(data=[[svc.score(x_test, y_test), dt.score(x_test, y_test), lg.score(x_test, y_test)]], columns=columns)
results = results.append(results_append, ignore_index = True)

In [42]:
results

,SVM,DecisionTree,LogisticRegression
0,0.976331,0.954142,0.974359
1,0.972387,0.954635,0.968442


In [43]:
pre_by_svm = svc.predict(test)
pre_by_dt = dt.predict(test)
pre_by_lg = lg.predict(test)
data2['label_by_svm'] = pre_by_svm
data2['label_by_dt'] = pre_by_dt
data2['label_by_lg'] = pre_by_lg

In [44]:
data2

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label_by_svm,label_by_dt,label_by_lg
0,0.247041,0.038041,0.256631,0.237936,0.268528,0.030592,2.467140,8.386868,0.800842,0.161888,...,0.048830,0.279070,2.132314,0.023438,12.398438,12.375000,0.147254,0,0,0
1,0.203363,0.060087,0.227414,0.136079,0.254168,0.118089,1.375516,3.853906,0.877562,0.234247,...,0.047105,0.269663,1.019271,0.023438,6.281250,6.257812,0.128839,1,1,1
2,0.186225,0.055964,0.201320,0.125506,0.235085,0.109579,1.635029,5.440385,0.910115,0.396274,...,0.047619,0.277457,0.735352,0.023438,2.671875,2.648438,0.118221,1,1,1
3,0.176482,0.058527,0.184521,0.128199,0.210805,0.082605,1.750256,6.873995,0.922014,0.453263,...,0.047572,0.277457,0.701891,0.023438,5.906250,5.882812,0.099374,1,1,1
4,0.159023,0.065191,0.175968,0.094592,0.196359,0.101767,3.229614,15.630528,0.896248,0.406344,...,0.020877,0.185185,0.414632,0.004883,0.776367,0.771484,0.582566,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628,0.214414,0.058021,0.226703,0.198510,0.254897,0.056386,1.722985,5.337870,0.893251,0.391982,...,0.048338,0.279070,0.890147,0.023438,5.507812,5.484375,0.123832,0,0,0
629,0.178708,0.059930,0.191608,0.115294,0.220157,0.104863,2.173189,8.533463,0.902936,0.433659,...,0.047013,0.279070,1.054688,0.023438,5.156250,5.132812,0.095337,1,1,1
630,0.176697,0.055536,0.163516,0.141978,0.231209,0.089231,1.863723,6.664198,0.917068,0.474086,...,0.047291,0.268156,1.939328,0.023438,8.437500,8.414062,0.175164,1,1,1
631,0.185480,0.053120,0.199095,0.154070,0.218090,0.064020,1.526036,5.031443,0.905618,0.391769,...,0.050686,0.271186,1.523438,0.023438,7.523438,7.500000,0.175463,0,1,0


In [45]:
data2.to_csv('voice_test.csv', index = False)